In [1]:
import moldesign as mdt
import moldesign.units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
mdt.configure()

In [3]:
molecule = mdt.from_pdb('1yu8')

In [4]:
molecule
#molecule = mdt.Molecule([res for res in molecule.residues if res.type == 'protein'])

### Molecule: "1yu8" (600 atoms)

<a href="http://www.rcsb.org/pdb/explore.do?structureId=1yu8" target="_blank">HIGH-RESOLUTION CRYSTAL STRUCTURES OF VILLIN HEADPIECE AND MUTANTS WITH REDUCED F-ACTIN BINDING ACTIVITY.</a>

**Mass**: 8150.00 amu

**Formula**: C<sub>325</sub>N<sub>85</sub>O<sub>189</sub>S<sub>1</sub>

**Charge**: 0.0 elementary_charge

### Residues

| chain | protein | dna | rna | unknown | water | solvent |
|-|-|-|-|-|-|-|-|
| <b>X</b> | 64 |  |  |  | 92 |  ||

### Biopolymer chains

**X**: <span style="word-wrap:break-word;font-family:monospace"><span title="PRO10 (MISSING), chain X" style="color:Red">P</span><span title="THR11 (MISSING), chain X" style="color:Red">T</span><span title="LYS12 (MISSING), chain X" style="color:Red">K</span><span title="LEU13 (idx 0), chain X">L</span><span title="GLU14 (idx 1), chain X">E</span><span title="THR15 (idx 2), chain X">T</span><span title="PHE16 (idx 3), chain X">F</span><span title="PRO17 (idx 4), chain X">P</span><span title="LEU18 (idx 5), chain X">L</span><span title="ASP19 (idx 6), chain X">D</span><span title="VAL20 (idx 7), chain X">V</span><span title="LEU21 (idx 8), chain X">L</span><span title="VAL22 (idx 9), chain X">V</span><span title="ASN23 (idx 10), chain X">N</span><span title="THR24 (idx 11), chain X">T</span><span title="ALA25 (idx 12), chain X">A</span><span title="ALA26 (idx 13), chain X">A</span><span title="GLU27 (idx 14), chain X">E</span><span title="ASP28 (idx 15), chain X">D</span><span title="LEU29 (idx 16), chain X">L</span><span title="PRO30 (idx 17), chain X">P</span><span title="ARG31 (idx 18), chain X">R</span><span title="GLY32 (idx 19), chain X">G</span><span title="VAL33 (idx 20), chain X">V</span><span title="ASP34 (idx 21), chain X">D</span><span title="PRO35 (idx 22), chain X">P</span><span title="SER36 (idx 23), chain X">S</span><span title="ALA37 (idx 24), chain X">A</span><span title="LYS38 (idx 25), chain X">K</span><span title="GLU39 (idx 26), chain X">E</span><span title="ASN40 (idx 27), chain X">N</span><span title="HIS41 (idx 28), chain X">H</span><span title="LEU42 (idx 29), chain X">L</span><span title="SER43 (idx 30), chain X">S</span><span title="ASP44 (idx 31), chain X">D</span><span title="GLU45 (idx 32), chain X">E</span><span title="ASP46 (idx 33), chain X">D</span><span title="PHE47 (idx 34), chain X">F</span><span title="LYS48 (idx 35), chain X">K</span><span title="ALA49 (idx 36), chain X">A</span><span title="VAL50 (idx 37), chain X">V</span><span title="PHE51 (idx 38), chain X">F</span><span title="GLY52 (idx 39), chain X">G</span><span title="MET53 (idx 40), chain X">M</span><span title="THR54 (idx 41), chain X">T</span><span title="ARG55 (idx 42), chain X">R</span><span title="SER56 (idx 43), chain X">S</span><span title="ALA57 (idx 44), chain X">A</span><span title="PHE58 (idx 45), chain X">F</span><span title="ALA59 (idx 46), chain X">A</span><span title="ASN60 (idx 47), chain X">N</span><span title="LEU61 (idx 48), chain X">L</span><span title="PRO62 (idx 49), chain X">P</span><span title="LEU63 (idx 50), chain X">L</span><span title="TRP64 (idx 51), chain X">W</span><span title="LYS65 (idx 52), chain X">K</span><span title="GLN66 (idx 53), chain X">Q</span><span title="GLN67 (idx 54), chain X">Q</span><span title="ASN68 (idx 55), chain X">N</span><span title="LEU69 (idx 56), chain X">L</span><span title="LYS70 (idx 57), chain X">K</span><span title="LYS71 (idx 58), chain X">K</span><span title="GLU72 (idx 59), chain X">E</span><span title="LYS73 (idx 60), chain X">K</span><span title="GLY74 (idx 61), chain X">G</span><span title="LEU75 (idx 62), chain X">L</span><span title="PHE76 (idx 63), chain X">F</span></span>

In [5]:
molecule.draw()
#len(molecule.atoms)
u.femtoseconds

<Unit('fs')>

In [6]:
import parmed as med

In [7]:
mdmol = mdt.assign_forcefield(molecule)


Connecting to docker host at unix://var/run/docker.sock... ERROR


You can fix this either:
  1) interactively, in a notebook, by running `moldesign.configure()`, or,
  2) by modifying the configuration dictionary `moldesign.compute.config`, then
     running `moldesign.compute.reset_compute_engine()` to try again.


AttributeError: 'module' object has no attribute 'Client'

In [8]:
mdmol.ff.amber_params.prmtop.put('/tmp/tmp.prmtop')
parmedmol = med.load_file('/tmp/tmp.prmtop')

NameError: name 'mdmol' is not defined

In [9]:
#mdmol.draw()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155


In [11]:

# In order to simluate using LAMMPS, we need:
#   get molecule from pdb, cif, or name
#   assign forcefields
#   load parmed structure


# LAMMPS UNITS:
# For style real, these are the units:

# mass = grams/mole
# distance = Angstroms
# time = femtoseconds
# energy = Kcal/mole
# velocity = Angstroms/femtosecond
# force = Kcal/mole-Angstrom
# torque = Kcal/mole
# temperature = Kelvin
# pressure = atmospheres
# dynamic viscosity = Poise
# charge = multiple of electron charge (1.0 is a proton)
# dipole = charge*Angstroms
# electric field = volts/Angstrom
# density = gram/cm^dim

# See http://parmed.github.io/ParmEd/html/index.html for ParmEd units

def formatLammpsData(dataName):
    filePath = '/tmp/data.' + dataName
    lammps_data = open(filePath, 'w')

    # print description
    lammps_data.write("LAMMPS Description\r\n\r\n")
    lammps_data.write("{0} atoms\r\n" .format(len(parmedmol.atoms)))
    lammps_data.write("{0} bonds\r\n" .format(len(parmedmol.bonds)))
    lammps_data.write("{0} angles\r\n" .format(len(parmedmol.angles)))
    lammps_data.write("{0} dihedrals\r\n" .format(len(parmedmol.dihedrals)))
    lammps_data.write("{0} impropers\r\n" .format(len(parmedmol.impropers)))
    
    lammps_data.write("\r\n")
    
    max_idx = 0
    for nonbond_idx in parmedmol.LJ_types.itervalues():
        if(nonbond_idx > max_idx):
            max_idx = nonbond_idx

    lammps_data.write("{0} atom types\r\n" .format(max_idx))
    lammps_data.write("{0} bond types\r\n" .format(len(parmedmol.bond_types)))
    lammps_data.write("{0} angle types\r\n" .format(len(parmedmol.angle_types)))
    lammps_data.write("{0} dihedral types\r\n" .format(len(parmedmol.dihedral_types)))
    lammps_data.write("{0} improper types\r\n" .format(len(parmedmol.improper_types)))
    
    lammps_data.write("\r\n")

    # box size
    # calculate lo and hi points
    xlo = xhi = ylo = yhi = zlo = zhi = None
    for atom in mdmol.atoms:
        if xlo == None:
            xlo = atom.x.value_in(u.angstrom)
            xhi = atom.x.value_in(u.angstrom)
            ylo = atom.y.value_in(u.angstrom)
            yhi = atom.y.value_in(u.angstrom)
            zlo = atom.z.value_in(u.angstrom)
            zhi = atom.z.value_in(u.angstrom)
        else:
            xlo = min(xlo, atom.x.value_in(u.angstrom))
            xhi = max(xhi, atom.x.value_in(u.angstrom))
            ylo = min(ylo, atom.y.value_in(u.angstrom))
            yhi = max(yhi, atom.y.value_in(u.angstrom))
            zlo = min(zlo, atom.z.value_in(u.angstrom))
            zhi = max(zhi, atom.z.value_in(u.angstrom))

    lammps_data.write("{0} {1} xlo xhi\r\n" .format(xlo, xhi))
    lammps_data.write("{0} {1} ylo yhi\r\n" .format(ylo, yhi))
    lammps_data.write("{0} {1} zlo zhi\r\n" .format(zlo, zhi))
    
    lammps_data.write("\r\n")

    # print masses -- Number of atom types
    lammps_data.write("Masses\r\n\r\n")
    for i in range(1, max_idx+1):
        for atom in parmedmol.atoms:
            if atom.nb_idx == i:
                lammps_data.write("{0} {1}\r\n" .format(i, atom.mass))
                break
    
    lammps_data.write("\r\n")
    

    # Pair Coeffs -- Number of atom types
    lammps_data.write("Pair Coeffs\r\n\r\n")
    for i in range(1, max_idx+1):
        for atom in parmedmol.atoms:
            if atom.nb_idx == i:
                lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(i, atom.epsilon, atom.sigma, 
                                                                    atom.epsilon_14, atom.sigma_14))
                break
    
    lammps_data.write("\r\n")
    

    # Bond Coeffs - Harmonic
    if parmedmol.bond_types:
        lammps_data.write("Bond Coeffs\r\n\r\n")
        for bt in parmedmol.bond_types:
            lammps_data.write("{0} {1} {2}\r\n" .format(bt.idx+1, bt.k, bt.req))
        
        lammps_data.write("\r\n")


    # Angle Coeffs - Harmonic
    if parmedmol.angle_types:
        lammps_data.write("Angle Coeffs\r\n\r\n")
        for at in parmedmol.angle_types:
            lammps_data.write("{0} {1} {2}\r\n" .format(at.idx+1, at.k, at.theteq))
       
        lammps_data.write("\r\n")


    # Dihedral Coeffs - Charmm
    if parmedmol.dihedral_types:
        lammps_data.write("Dihedral Coeffs\r\n\r\n")
        for dt in parmedmol.dihedral_types:
            lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(dt.idx+1, dt.phi_k, dt.per, 
                (180 if dt.phase >= 180 else 0), 
                (1.0 if dt.scnb == 1.0 else 0.0)))
        
        lammps_data.write("\r\n")


    # Improper Coeffs - Harmonic
    if parmedmol.improper_types:
        lammps_data.write("Improper Coeffs\r\n\r\n")
        for it in parmedmol.improper_types:
            print
            lammps_data.write("{0} {1} {2}\r\n" .format(it.idx+1, it.psi_k, it.psi_eq))
        
        lammps_data.write("\r\n")


    # print atoms
    lammps_data.write("Atoms\r\n\r\n")
    for atom in parmedmol.atoms:
        lammps_data.write("{0} {1} {2} {3} " .format(atom.idx+1, atom.residue.idx, atom.nb_idx, atom.charge))
        if not mdmol.positions.any():        
            lammps_data.write("{0} {1} {2} " .format(0.00, 0.00, 0.00))        
        else:
            pos = mdmol.positions[atom.idx]
            lammps_data.write("{0} {1} {2} " .format(pos[0].value_in(u.angstrom), pos[1].value_in(u.angstrom), 
                pos[2].value_in(u.angstrom)))
        lammps_data.write("{0} {1} {2}\r\n" .format(0, 0, 0)) # nx, ny, nz not defined    

    lammps_data.write("\r\n")


    # print velocities
    lammps_data.write("Velocities\r\n\r\n")
    for atom in mdmol.atoms:
        xvel = atom.vx.value_in(u.angstrom/u.fs)
        yvel = atom.vy.value_in(u.angstrom/u.fs)
        zvel = atom.vz.value_in(u.angstrom/u.fs)
        lammps_data.write("{0} {1} {2} {3}\r\n" .format(atom.index+1, xvel, yvel, zvel))
        
    lammps_data.write("\r\n")


    # print bonds
    lammps_data.write("Bonds\r\n\r\n")
    for idx, bond in enumerate(parmedmol.bonds):
        lammps_data.write("{0} {1} {2} {3}\r\n" .format(idx+1, bond.type.idx+1, 
          bond.atom1.idx+1, bond.atom2.idx+1))
        
    lammps_data.write("\r\n")


    # print angles
    lammps_data.write("Angles\r\n\r\n")
    for idx, angle in enumerate(parmedmol.angles):
        lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(idx+1, angle.type.idx+1,
          angle.atom1.idx+1, 
          angle.atom2.idx+1, angle.atom3.idx+1))
        
    lammps_data.write("\r\n")


    # print dihedrals
    if parmedmol.dihedrals:
        lammps_data.write("Dihedrals\r\n\r\n")
        for idx, di in enumerate(parmedmol.dihedrals):
            lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(idx+1, di.type.idx+1, 
              di.atom1.idx+1, di.atom2.idx+1, 
              di.atom3.idx+1, di.atom4.idx+1))
            
        lammps_data.write("\r\n")


    # print impropers
    if parmedmol.impropers:
        lammps_data.write("Impropers\r\n\r\n")
        for im in parmedmol.impropers:
            lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(im.idx+1, im.type.idx+1, 
              im.atom1.idx+1, im.atom2.idx+1, 
              im.atom3.idx+1, im.atom4.idx+1))
        
        lammps_data.write("\r\n")

    
    lammps_data.close()
    return filePath

In [12]:
name = formatLammpsData('test')

In [13]:
def prepareLammps(dataPath):
    L.command("units real")
    L.command("atom_style full")
    L.command("pair_style lj/charmm/coul/long 8.0 10.0 10.0")
    L.command("bond_style harmonic")
    L.command("angle_style harmonic")
    L.command("dihedral_style charmm")
    L.command("improper_style harmonic")
    L.command("kspace_style pppm 0.0001")
    L.command("read_data " + dataPath)

def calculateEnergy(timeStep, outputFreq):
    #L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    #L.command("thermo " + str(outputFreq))

            
def groupHydrogens():
    groupCommand = "group hbonds type"
    for nonbond_name, nonbond_idx in parmedmol.LJ_types.iteritems():
        if(nonbond_name[0] == 'H'):
            groupCommand = groupCommand + " " + str(nonbond_idx)
    L.command(groupCommand)
    
    
def runNvt(timeStep, temp, damp, outputFreq, applyShake):
    L.command("neighbor 2.0 bin")
    #L.command("neigh_modify delay 5")
    L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    L.command("thermo " + str(outputFreq))
    nvt_command = "fix 1 all nvt temp {0} {1} {2}" .format(round(temp,1), round(temp,1), round(damp, 3))
    L.command(nvt_command)
    if(applyShake):
        groupHydrogens()
        shake_command = "fix 2 hbonds rattle 0.0001 20 0 t 5 6 m 1.0 a 31"
        L.command(shake_command)


In [14]:
from lammps import lammps, PyLammps

In [15]:
lmp = lammps()
L = PyLammps(ptr=lmp)

LAMMPS output is captured by PyLammps wrapper


In [16]:
prepareLammps(name)

In [17]:
print "Before Evaluation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

Before Evaluation
(8.877, 11.103, 14.169)
(0.0, 0.0, 0.0)
(8.487983166e-314, 8.487983166e-314, 8.487983166e-314)


In [18]:
calculateEnergy(1.0, 10)
L.run(1000)
print L.eval("pe")


-4179.44402414


In [19]:
print L.atoms.natoms
#for i in range(0, L.atoms.natoms):
    #print L.atoms[i].force

1289


In [20]:
print "After Evaluation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

After Evaluation
(8.877, 11.103, 14.169)
(0.0, 0.0, 0.0)
(-7.6358182133318175, -7.891001958914227, 45.069846815993344)


In [21]:
del L
del lmp

lmp = lammps()
L = PyLammps(ptr=lmp)

LAMMPS output is captured by PyLammps wrapper


In [22]:
prepareLammps(name)

In [23]:
print "Before Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

Before Simulation
(8.877, 11.103, 14.169)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)


In [24]:
runNvt(1.0, 300, 100, 10, True)


In [25]:
# shake_command = "fix 2 hbonds shake 0.0001 10 0 b 4 6 8 10 12 14 18 a 31"
# L.command(shake_command)


In [26]:
L.run(200)

[u'PPPM initialization ...',
 u'WARNING: System is not charge neutral, net charge = -2 (../kspace.cpp:298)',
 u'WARNING: Using 12-bit tables for long-range coulomb (../kspace.cpp:317)',
 u'  G vector (1/distance) = 0.243816',
 u'  grid = 15 15 12',
 u'  stencil order = 5',
 u'  estimated absolute RMS force accuracy = 0.0251739',
 u'  estimated relative force accuracy = 7.58106e-05',
 u'  using double precision FFTs',
 u'  3d grid and FFT values/proc = 6800 2700',
 u'Neighbor list info ...',
 u'  1 neighbor list requests',
 u'  update every 1 steps, delay 10 steps, check yes',
 u'  max neighbors/atom: 2000, page size: 100000',
 u'  master list distance cutoff = 12',
 u'  ghost atom cutoff = 12',
 u'  binsize = 6, bins = 7 6 6',
 u'Setting up Verlet run ...',
 u'  Unit style    : real',
 u'  Current step  : 0',
 u'  Time step     : 1',
 u'Memory usage per processor = 19.8786 Mbytes',
 u'Step Temp PotEng TotEng ',
 u'       0            0   -4179.4486   -4179.4486 ',
 u'      10    144.04

In [27]:
print "After Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

After Simulation
(9.19494075859492, 11.451470715040097, 15.346603769920547)
(-0.00397251412118858, -0.0015766077350572762, -0.010748494668691991)
(0.9883842516969316, -14.583172882856926, 8.907208281244325)


In [28]:
#

(7.788610126744418, 10.530872852534074, 14.642204100255647)
(-0.007559879676876478, -0.001795016071579182, 0.0026680477474389707)
(-2.1826713735872714, 14.545634070314566, 3.0004310601325535)


In [29]:
# mdmol.draw3d() 17

In [30]:
for j in range(0, 10):
    L.run(200)
    print "After Simulation"
    print L.atoms[1012].position
    print L.atoms[1012].velocity
    print L.atoms[1012].force
    

After Simulation
(7.457580848092356, 10.69993265047156, 14.956169430314278)
(0.005556983376549892, 0.0018551796995334708, -3.141111672232247e-05)
(2.191356380785142, -6.627433362346804, -1.5312295043057897)
After Simulation
(7.721168668859181, 10.97800544000138, 14.947162795601557)
(0.0006144565146105085, -4.639190940648405e-05, -0.0008430402012587128)
(2.8094228602161753, 1.7414929341973215, -7.386455313419221)
After Simulation
(7.869852491935284, 10.936239989486067, 14.811540042196986)
(0.00012977984968271205, -0.000522515180828844, 0.0009188572408656176)
(3.761312657263911, -1.0842196733355531, -1.9149606833162571)
After Simulation
(7.871100018519319, 10.926784103576775, 14.82326176218348)
(-0.0005161291641022777, 0.00010222442986587411, 9.83790687218567e-05)
(-0.15044681444352428, 0.09599551605240592, -1.8462554568758622)
After Simulation
(7.893233854832725, 10.901471274266948, 14.664484256308727)
(-0.00045665182134537233, -4.522541275381013e-07, -0.0007110559481191951)
(0.38737094

In [31]:
L.run(10)

[u'PPPM initialization ...',
 u'WARNING: Using 12-bit tables for long-range coulomb (../kspace.cpp:317)',
 u'  G vector (1/distance) = 0.243816',
 u'  grid = 15 15 12',
 u'  stencil order = 5',
 u'  estimated absolute RMS force accuracy = 0.0251739',
 u'  estimated relative force accuracy = 7.58106e-05',
 u'  using double precision FFTs',
 u'  3d grid and FFT values/proc = 6800 2700',
 u'Setting up Verlet run ...',
 u'  Unit style    : real',
 u'  Current step  : 2400',
 u'  Time step     : 1',
 u'Memory usage per processor = 19.8786 Mbytes',
 u'Step Temp PotEng TotEng ',
 u'    2400      348.907   -6404.9213   -5097.2626 ',
 u'    2410    391.39516   -6552.3865   -5085.4876 ',
 u'Loop time of 0.081291 on 1 procs for 10 steps with 1289 atoms',
 u'',
 u'Performance: 10.628 ns/day, 2.258 hours/ns, 123.015 timesteps/s',
 u'99.2% CPU use with 1 MPI tasks x no OpenMP threads',
 u'',
 u'MPI task timing breakdown:',
 u'Section |  min time  |  avg time  |  max time  |%varavg| %total',
 u'-----

In [34]:
for j in range(0, 10):
    L.run(1)
    print "After Simulation"
    print L.atoms[1012].position
    print L.atoms[1012].velocity
    print L.atoms[1012].force

After Simulation
(8.6383729709806, 11.997737928448526, 14.997921995573893)
(0.0016438358961545381, -0.00013722269212715254, -0.00384629634136644)
(-0.19733122030875017, -0.9099831571712238, -6.018102449696341)
After Simulation
(8.640014771306522, 11.997588762270365, 14.993995738422633)
(0.0016416456888421202, -0.00016147075696782386, -0.004001437557920804)
(-0.05470652367141737, -0.9368591754884609, -5.645418413058776)
After Simulation
(8.64165627814647, 11.99741498538168, 14.989919081975053)
(0.001643390994006622, -0.00018684691407759506, -0.0041452705933842215)
(0.09952048904172772, -0.9945506178215493, -5.135279988380687)
After Simulation
(8.643301550038975, 11.997215068812354, 14.985705205447646)
(0.0016493852658386338, -0.00021456246321972919, -0.004274999485737021)
(0.2708260405446806, -1.1144506707521866, -4.561343737586456)
After Simulation
(8.644955033827605, 11.996985862387092, 14.981369121494316)
(0.001659805378445211, -0.00024627583565876133, -0.004389828051340082)
(0.44166

In [39]:
print L.atoms.natoms
i = 0
for atom in mdmol.velocities:
    i+=1
print i



1289
1289
